In [ ]:
# This model is optimized for GPU but not tested stability

In [1]:
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

from ogb.graphproppred import PygGraphPropPredDataset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.data import DataLoader

from logger import Logger

In [3]:
device_args = 0 #default
device = f'cuda:{device_args}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [4]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor())
split_idx = dataset.get_idx_split()
data = dataset[0]

data.adj_t = data.adj_t.to_symmetric()
data = data.to(device)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [6]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)

In [8]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [13]:
num_layers = 3
hidden_channels = 256
dropout = 0.5
lr = 0.1
epochs = 500
runs = 10
log_steps = 1

model = GCN(data.num_features, hidden_channels, dataset.num_classes, num_layers, dropout).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [14]:
for run in range(runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        for epoch in range(1, 1 + epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, split_idx, evaluator)
            # logger.add_result(run, result)

            if epoch % log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

Run: 01, Epoch: 01, Loss: 4.1986, Train: 10.99%, Valid: 22.97% Test: 21.56%
Run: 01, Epoch: 02, Loss: 5.6963, Train: 5.85%, Valid: 4.25% Test: 3.06%
Run: 01, Epoch: 03, Loss: 6.2524, Train: 7.69%, Valid: 14.96% Test: 22.10%
Run: 01, Epoch: 04, Loss: 4.5490, Train: 9.20%, Valid: 11.15% Test: 10.49%
Run: 01, Epoch: 05, Loss: 4.0974, Train: 19.84%, Valid: 9.90% Test: 7.83%
Run: 01, Epoch: 06, Loss: 3.5769, Train: 25.59%, Valid: 28.95% Test: 25.90%
Run: 01, Epoch: 07, Loss: 3.3349, Train: 20.43%, Valid: 26.36% Test: 23.90%
Run: 01, Epoch: 08, Loss: 3.1642, Train: 14.89%, Valid: 24.34% Test: 22.41%
Run: 01, Epoch: 09, Loss: 2.9893, Train: 16.32%, Valid: 24.80% Test: 22.74%
Run: 01, Epoch: 10, Loss: 2.8215, Train: 23.55%, Valid: 27.76% Test: 24.98%
Run: 01, Epoch: 11, Loss: 2.6984, Train: 27.80%, Valid: 29.99% Test: 26.84%
Run: 01, Epoch: 12, Loss: 2.6269, Train: 29.72%, Valid: 31.40% Test: 28.05%
Run: 01, Epoch: 13, Loss: 2.5823, Train: 30.40%, Valid: 32.15% Test: 28.80%
Run: 01, Epoch: 14,

KeyboardInterrupt: 